In [ ]:
import pandas as pd
import numpy as np
import statistics as stat
from datetime import datetime, date
from sklearn.cluster import KMeans
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import norm, shapiro

import scipy.stats as st
from scipy.stats import ttest_ind
import random

import pandas as pd
import pyodbc


In [ ]:

def normality_test(x):
    try:
        # stat , p = shapiro(x)
        stat, p = stats.normaltest(x)
    except:
        return "small sample"
    return "rejected" if p < 0.05 else "not rejected"


def shapiro_test(x):
    try:
        # stat , p = shapiro(x)
        stat, p = shapiro(x)
    except:
        return "small sample"
    return "rejected" if p < 0.05 else "not rejected"


def kolmagorov_test(x):
    try:
        # stat , p = shapiro(x)
        stat, p = stats.kstest(x, 'norm')
    except:
        return "small sample"
    return "rejected" if p < 0.05 else "not rejected"


filters = ['organization', 'temprature_rnd', 'month']
desc_group = data_volume.groupby(filters).agg({'volume': [
    'mean', 'std', 'count', 'median', normality_test, shapiro_test, kolmagorov_test]})
desc_group.columns = desc_group.columns.droplevel(0)
# desc_group.loc[(ranges_orgs,5),:]
result = desc_group[["normality_test", "shapiro_test", "kolmagorov_test"]]
result = result.reset_index()
result.melt(id_vars=['organization', 'temprature_rnd'],
            value_vars=["normality_test", "shapiro_test", "kolmagorov_test"],
            var_name='method', value_name='test').to_excel("result_hypo.xlsx")

sns.histplot(vol_data, kde=True, binwidth=std, edgecolor='black')
print(len(vol_data))
plt.show()




In [ ]:

# Set a correct value for num_tests_50percent
num_tests_50percent = None


# Create the plot
sig_threshold = 0.05
num_tests = np.array(range(50))
probabilities = 1-((1-sig_threshold)**num_tests)
plt.plot(num_tests, probabilities)
# Edit title and axis labels
plt.title('Type I Error Rate for Multiple Tests', fontsize=15)
# Label the y-axis
plt.ylabel('Probability of at Least One Type I Error', fontsize=12)
# Label the x-axis
plt.xlabel('Number of Tests', fontsize=12)


In [ ]:

plot_table = data_volume_control
profiles = list(set(plot_table.profile))
profile = profiles[5]
version = ''
x_name, y_name = "temprature", "volume"
filename = 'analysis_KG ' + str(profile) + \
    version + "_" + x_name + "-" + y_name + '.pdf'
y = np.array(plot_table[plot_table.profile == profile][y_name])
x = np.array(plot_table[plot_table.profile == profile][x_name])

fig = plt.figure(figsize=(50, 20))
ax = fig.add_subplot(1, 2, 1)
ax.scatter(x, y, s=10)

plt.xticks(np.arange(min(x), max(x), 2))
plt.xticks(fontsize=8, rotation=90)
plt.yticks(np.arange(min(y), max(y), max(y)/50))
# ax.plot(x_approx, y_approx, label = days + '_approx', color = '#006400')
plt.ylabel(y_name)
plt.xlabel(x_name)
plt.title("Профіль " + str(profile), fontsize=15)
plt.savefig(filename)
plt.show()
variance_coeff = print(stat.stdev(y)/np.mean(y))
variance_coeff

sns.histplot(array)
plt.show()



In [ ]:
# колонка с количеством дней и вывоод фильтра температуры

data_volume_control = data_volume[data_volume.peracc.isin(array)]
data_volume_control.date = pd.to_datetime(
    data_volume_control.date, dayfirst=True)
data_volume_control["number_of_day"] = data_volume_control.date.apply(
    lambda x: x.timetuple().tm_yday)
data_volume_control[['volume', 'temprature']]
data_volume_control.loc[:, '2darray'] = data_volume_control.apply(
    lambda x: [x.volume, x.temprature], axis=1)

data_volume_control_by_temp = data_volume_control[(data_volume_control.temprature < 22) &
                                                  (data_volume_control.volume > 0)]


In [ ]:

y = np.array(list(plot_table['2darray']))
distortions = []
for i in range(1, 11):
    km = KMeans(
        n_clusters=i, init='random',
        n_init=10, max_iter=300,
        tol=1e-04, random_state=0
    )
    km.fit(y)
    distortions.append(km.inertia_)

# plot
plt.plot(range(1, 11), distortions, marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Distortion')
plt.show()


In [ ]:
kmeans = KMeans(n_clusters=6)
kmeans.fit(y)
y_kmeans = kmeans.predict(y)


In [ ]:
plt.scatter(y[:, 1], y[:, 0], c=y_kmeans, s=20, cmap='summer')
centers = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='blue', s=100, alpha=0.9)

plt.show()


In [ ]:
data_classified = pd.concat([pd.Series(list(y_kmeans)),
                             pd.Series(y.tolist()),
                             plot_table["profile"].reset_index(drop=True)], axis=1)
data_classified.columns = ["new_class", "intervals", "profile"]


In [ ]:
pd.crosstab(data_classified.new_class, data_classified.profile,
            margins=True, margins_name="Total")
crossing = pd.crosstab(data_classified.new_class,
                       data_classified.profile, normalize='index').round(2)
crossing["max"] = crossing.max(axis=1)


In [ ]:

plt.rcParams.update({'font.size': 5})
fig = plt.figure(figsize=(14, 6), dpi=80)
fig.subplots_adjust(hspace=0.5)

for new_class in list(range(6)):
    y_cond = np.array(
        data_classified[data_classified.new_class == new_class].intervals.to_list())
    y_kmeans = np.array(
        data_classified[data_classified.new_class == new_class].new_class)
    ax = fig.add_subplot(4, 4, 4)
    ax.scatter(y_cond[:, 1], y_cond[:, 0], c=y_kmeans, s=20, cmap='summer')
    ax.scatter(centers[new_class][:, 0], centers[new_class]
               [:, 1], c='blue', s=100, alpha=0.9)
# plt.savefig("clusterization.pdf")
plt.show()
